In [1]:
%pip install "scikit-learn<1.6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/11_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [2]:
%pip install lightgbm

In [3]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00


In [4]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.ensemble import RandomForestRegressor

stacking_reg = StackingRegressor(
    estimators=[
        ('RandomForest', RandomForestRegressor(n_estimators=50, max_depth=8, random_state=42)),
        ('lr', LinearRegression()),
        ('LGBM', LGBMRegressor(n_estimators=111, max_depth=11, random_state=42)),
        ('xgboost', xgb.XGBRegressor(n_estimators=51, max_depth=8, random_state=42)),
        ('catboost', Catboost(n_estimators=56, depth=7, learning_rate=1, random_state=42)),
    ],
    final_estimator=LinearRegression(),
    n_jobs=-1,
    cv=3
)

stacking_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


StackingRegressor(cv=3,
                  estimators=[('RandomForest',
                               RandomForestRegressor(max_depth=8,
                                                     n_estimators=50,
                                                     random_state=42)),
                              ('lr', LinearRegression()),
                              ('LGBM',
                               LGBMRegressor(max_depth=11, n_estimators=111,
                                             random_state=42)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=8,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=51, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x79f8f3b30340>)],
                  final_estimator=LinearRegression(), n_jobs=-1)

In [5]:
from sklearn.metrics import root_mean_squared_error
y_pred = stacking_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.044614877475177

In [6]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [7]:
y_pred = stacking_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0521982651325315

In [8]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/11_test.csv')
del test_set['Premium Amount']
y_pred = stacking_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('stacking_reg_16.csv', index=False)